Instructions:

- Run build_example_data.ipynb, which will populate rental_car_time_series.csv in the example_use directory
- Run this notebook

Notes:
- This notebook is designed to be run locally, and as such, cannot utilize the parallel pipeline. 
  - Sample code to run the parallel pipeline has been inserted and commented out. 
  - In order to use the parallel pipeline, TEMPO requires PySpark and the ability to write to databricks tables

# Setup

## Imports

In [0]:
import pandas as pd
import numpy as np

import datetime
import pytz

from tempo_forecasting.utils.training_utils import calculate_time_periods

from tempo_forecasting.utils.logging_utils import logger
from tempo_forecasting.pipeline.parallel_pipeline import ParallelPipeline

from tempo_forecasting.pipeline.preprocessing_pipeline import preprocess_pipeline
from tempo_forecasting.pipeline.training_pipeline import train_pipeline
from tempo_forecasting.pipeline.evaluation_pipeline import evaluation_pipeline

from tempo_forecasting.utils.plotting_utils import plot_side_by_side
from tempo_forecasting.optuna_opt.run_optuna import OptunaConfig

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [0]:
np.random.seed(42)

pd.set_option('display.max_colwidth', None)

## Data Read

In [0]:
demand_df = pd.read_csv("./rental_car_time_series.csv")
demand_df.head(5)

In [0]:
min_date = demand_df["date"].min()
max_date = pd.Timestamp(demand_df["date"].max())
print(min_date,max_date)

## Arg Assignment

In [0]:
date_config = {
    "max_date": str(max_date)[:10],
    "n_test_months": 6,
    "n_train_months": 36,
    "n_validation_sets": 4,
    "cv_window_step_days": 90
}

time_periods = calculate_time_periods(max_date_str = str(date_config['max_date']),
                                      n_test_months = date_config["n_test_months"],
                                      n_train_months = date_config["n_train_months"],
                                      n_validation_sets = date_config["n_validation_sets"],
                                      cv_window_step_days = date_config["cv_window_step_days"],
                                      verbose = True)

In [0]:
args = {
    'date_col': 'date',
    'target_y': 'n_rented',
    'freq': 'D',
    'cv_dates': [[d["min"],d["cutoff"],d["max"]] for d in time_periods["cv_windows"]],
    'retrain_dates': [time_periods["retrain_range"]["min"],time_periods["retrain_range"]["max"]],
    'use_parallel': False,
    'min_target_value': 5, # min target y value in time series to consider models outside moving average 
    'future_forecast_horizon': 365
    }

logger.info(f"Initializing model args: {args}")

n_trials = 3
optuna_config = OptunaConfig(
    n_trials = n_trials,
    n_startup_trials = 1,
    timeout_sec = 15 * n_trials * date_config["n_validation_sets"], # allocate 10 seconds per trial as program timeout
    eval_metric = "wmape"
)

group_col = 'category'

## Helper Functions

In [0]:
def plot_init_model_charts(category,all_date_vals,param_df):
    tmp_params = param_df[param_df["category"]==category]

    best_metric_type = tmp_params["metric_type"][0]
    best_metric_avg_val = round(tmp_params["cv_avg_metric"][0],4)

    p = tmp_params["best_params"][0]

    line_limit = 150
    if len(p)>line_limit:
        chunked_p = []
        current_line = ""
        for s in p.split(","):
            if len(current_line) + len(s) <= line_limit:
                current_line += f"{s}, "
            else:
                chunked_p.append(f"{current_line}")
                current_line = s
        chunked_p.append(current_line)

        p = "\n".join(chunked_p)

    cap_add = f"; Cross-Validation Avg. {best_metric_type} = {best_metric_avg_val}\n{p}"

    plot_side_by_side(category,all_date_vals,param_df,caption_add=cap_add)

In [0]:
def get_base_colors():
    # Create a color palette with enough colors for all model types
    # Using a mix of qualitative colors that work well for both regular and vivid versions
    base_colors = [
        '#1f77b4',  # Blue
        '#ff7f0e',  # Orange
        '#2ca02c',  # Green
        '#d62728',  # Red
        '#9467bd',  # Purple
        '#8c564b',  # Brown
        '#e377c2',  # Pink
        '#7f7f7f',  # Gray
        '#bcbd22',  # Olive
        '#17becf',  # Cyan
        '#aec7e8',  # Light blue
        '#ffbb78',  # Light orange
        '#98df8a',  # Light green
        '#ff9896',  # Light red
        '#c5b0d5',  # Light purple
    ]

    return base_colors

def plot_model_performance(df, 
                           figsize=(8, 6), 
                           alpha_regular=0.4, 
                           alpha_best=0.9, 
                           marker_size_regular=80, 
                           marker_size_best=50,
                           logscale=False,
                           title="Model Performance: MAE vs WMAPE"):
    """
    Create a scatter plot of model performance metrics (MAE vs WMAPE).
    
    Parameters:
    df (pd.DataFrame): DataFrame with columns ['category', 'model_type', 'WMAPE', 'MAE', 'is_best_model']
    figsize (tuple): Figure size as (width, height)
    alpha_regular (float): Transparency for regular models (0-1)
    alpha_best (float): Transparency for best models (0-1)
    marker_size_regular (int): Size of regular model markers
    marker_size_best (int): Size of best model markers
    logscale (bool): whether to apply a logarithmic scale to the axes
    title (str): Plot title
    
    Returns:
    fig, ax: Matplotlib figure and axis objects
    """
    
    # Validate input DataFrame
    required_columns = ['category', 'model_type', 'WMAPE', 'MAE', 'is_best_model']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"DataFrame must contain columns: {required_columns}")
    
    if df.empty:
        raise ValueError("DataFrame cannot be empty")
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=figsize)
    
    # Get unique model types
    model_types = df['model_type'].unique()
    
    # Extend colors if we have more model types than base colors
    base_colors = get_base_colors()
    colors = base_colors * (len(model_types) // len(base_colors) + 1)
    
    # Create color mapping for model types
    model_colors = {model_type: colors[i] for i, model_type in enumerate(model_types)}
    
    # Separate data into best models and regular models
    best_models = df[df['is_best_model'] == True]
    regular_models = df[df['is_best_model'] == False]
    
    # Plot regular models first (so best models appear on top)
    for model_type in model_types:
        regular_subset = regular_models[regular_models['model_type'] == model_type]
        if not regular_subset.empty:
            ax.scatter(regular_subset['WMAPE'], regular_subset['MAE'], 
                      color=model_colors[model_type], alpha=alpha_regular, 
                      s=marker_size_regular, label=f'{model_type} (regular)',
                      edgecolors= None, linewidth=0.5)
    
    # Plot best models with vivid colors
    for model_type in model_types:
        best_subset = best_models[best_models['model_type'] == model_type]
        if not best_subset.empty:
            ax.scatter(best_subset['WMAPE'], best_subset['MAE'], 
                      color=model_colors[model_type], alpha=alpha_best, 
                      s=marker_size_best, label=f'{model_type} (best)',
                      edgecolors=None, linewidth=1.5, marker='D')  # Diamond shape for best
    
    # Customize the plot
    ax.set_xlabel('WMAPE (Weighted Mean Absolute Percentage Error)', fontsize=12)
    ax.set_ylabel('MAE (Mean Absolute Error)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)

    if logscale:
        ax.set_xscale('log')
        ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax.set_yscale('log')
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
    
    # Add legend
    # Create a cleaner legend by grouping model types
    handles, labels = ax.get_legend_handles_labels()
    
    # Create custom legend with better organization
    legend_elements = []
    for model_type in model_types:
        # Add entry for regular models
        if any(f'{model_type} (regular)' in label for label in labels):
            legend_elements.append(plt.scatter([], [], color=model_colors[model_type], 
                                             s=marker_size_regular, alpha=alpha_regular, 
                                             edgecolors= None, linewidth=0.5,
                                             label=f'{model_type}'))
        # Add entry for best models
        if any(f'{model_type} (best)' in label for label in labels):
            legend_elements.append(plt.scatter([], [], color=model_colors[model_type], 
                                             s=marker_size_best, alpha=alpha_best, 
                                             edgecolors= None, linewidth=1.5, marker='D',
                                             label=f'{model_type} (best)'))
    
    # Place legend outside the plot area
    ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Add some styling improvements
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('gray')
    ax.spines['bottom'].set_color('gray')
    
    # Add text annotation with summary statistics
    total_models = len(df)
    best_models_count = len(best_models)
    unique_categories = df['category'].nunique()
    
    stats_text = f'Total Models: {total_models}\nCategories: {unique_categories}'
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Adjust layout to prevent legend cutoff
    plt.tight_layout()
    
    return fig, ax

# Modeling 

In [0]:
if args['use_parallel']:
    print("Parallel only supported in DataBricks via PySpark currently, sorry.")
    # run_timestamp = datetime.datetime.now(pytz.timezone('America/Chicago')).strftime("%Y%m%d-%H%M%S")
    # print(f"JOB RUN_DATETIME: {run_timestamp}")

    # # group_sublist = [] #if you want to select a subset of categories to model

    # # demand_df = modeling_df.where(F.col("category_group").isin(group_sublist))

    # parallel_pipeline = ParallelPipeline(run_id = run_timestamp,
    #                                     group_col = "category",
    #                                     args = args,
    #                                     optuna_config = optuna_config,
    #                                     target_metric = "WMAPE",
    #                                     catalog_name=f"dev_data_science"
    #                                     )

    # modeling_results = parallel_pipeline.run_parallel_forecasting(demand_df)

    # model_parameters_df, model_data_df, full_results_df = parallel_pipeline.postprocess_results(modeling_results)

else:
    results = {}
    param_dfs = []
    output_data_dfs = []

    categories = demand_df[group_col].unique()

    for category in categories:
        # Filter the DataFrame for the current category
        category_data = demand_df[demand_df[group_col]==category].drop(group_col, axis=1).copy()

        # Step 1: Preprocessing
        preproc_output = preprocess_pipeline(category = category,
                                                category_data = category_data, 
                                                args = args,
                                                logger = None)
        model_set, cv_dates, _ = preproc_output
        
        # Step 2: Training
        train_output = train_pipeline(category = category, 
                                        models = model_set, 
                                        modeling_data = category_data, 
                                        args = args, 
                                        optuna_config = optuna_config,
                                        override_cv_dates = cv_dates,
                                        logger = "print")   
        
        category_results, _ = train_output
        results[category] = category_results

        # # Step 3: Evaluation
        eval_output = evaluation_pipeline(category = category,
                                            category_results = category_results,
                                            args = args,
                                            target_metric = "WMAPE",
                                            forecast_horizon = args['future_forecast_horizon'],
                                            logger = "print")
        category_param_df, category_output_data_df, _ = eval_output

        param_dfs += [category_param_df]
        output_data_dfs += [category_output_data_df]

        # Build results json, slightly more complicated but same idea
        results_cols = ["category","model","cv_best_avg_metrics",
                        "cv_best_all_metrics","cv_all_trials_all_metrics","best_params"]
        results_df = pd.DataFrame(columns=results_cols)

        for cat in results:
            for model in results[cat]["models"]:
                cm_results = results[cat]["models"][model]

                new_row = {'category': cat, 
                            'model': model,
                            'cv_best_avg_metrics': cm_results["cv_metrics"]["cv_best_mean_all_metrics"],
                            'cv_best_all_metrics': cm_results["cv_metrics"]["cv_best_full_all_metrics"],
                            'cv_all_trials_all_metrics': cm_results["cv_metrics"]["cv_all_trials_all_metrics"],
                            'best_params': cm_results["model_params"],
                            }

                results_df = pd.concat([results_df,pd.DataFrame([new_row])],ignore_index=True)

        param_df = pd.concat(param_dfs)
        output_data_df = pd.concat(output_data_dfs)

# Standard Assessment

## Output Check

In [0]:
# Details of the final model chosen by Optuna
param_df

In [0]:
# In-depth details for all models run by optuna for each category. 
results_df

In [0]:
# Final time series values. Contains the true values, the fit and pred values associated with the last cv window, and the fit and forecast values associated with the final retrained model
output_data_df

## High Level Modeling Checks

### Optuna Trials per Model

Three different things may determine when an Optuna study is ended:
- Optuna has performed the intended number of trials, as specified in the optuna_config's n_trials parameter
- Optuna has surpassed the intended time limit for a study, as specified in the optuna_config's timeout_sec parameter
- The parameter search space was determined to be small enough for a grid search, and all possible parameter configurations were searched

Each model type is prone to different types of study end conditions. As such, it is helpful to report statistics on how many studies were actually performed

In [0]:
n_trials_df = results_df[["category", "model"]].copy()
n_trials_df["n_trials"] = results_df["cv_all_trials_all_metrics"].apply(
    lambda x: len(x["WMAPE"]) if x is not None and "WMAPE" in x else 0
)

# Group by model and calculate aggregations
result = n_trials_df.groupby("model")["n_trials"].agg([
    ("mean_n_trials", "mean"),
    ("std_n_trials", "std"),
    ("min_n_trials", "min"),
    ("med_n_trials", "median"),
    ("max_n_trials", "max")
]).round(2).T

result

### MAE vs. WMAPE by model type

WMAPE is generally preferred when comparing across differing scales of data, however, there is a tradeoff with MAE. 

For example, a model where only 10 cars are rented on average might show a high WMAPE (30%!), but MAE (3) would show that we forecasted 13 cars, which is more explainable.

The following chart allows systematic assessment of this trade off, color coded by model type and whether the model was selected as "best"

In [0]:
model_comparison_df = results_df[["category"]]
model_comparison_df["model_type"] = results_df["model"]
cv_best_all_metrics = results_df["cv_best_all_metrics"]

model_comparison_df["WMAPE"] = [round(np.mean(row["WMAPE"]),3) for row in cv_best_all_metrics]
model_comparison_df["MAE"] = [round(np.mean(row["MAE"]),3) for row in cv_best_all_metrics]

selected_models = param_df[["category"]]
selected_models["model_type"] = param_df["model_name"]
selected_models["is_best_model"] = True

model_comparison_df = model_comparison_df.merge(selected_models, on=["category","model_type"], how="left").fillna({"is_best_model":False})
model_comparison_df.head()

In [0]:
fig, ax = plot_model_performance(model_comparison_df, figsize=(9, 5), logscale=True)

In [0]:
# Single Category
cat = "Car A"

fig, ax = plot_model_performance(
    model_comparison_df[model_comparison_df["category"]==cat], 
    figsize=(9,5),
    alpha_regular=0.5,
    marker_size_best=60,
    title=f"{cat}: Model Performance Analysis"
)

## Individual Model Results
side by side plots with additional parameter details in the caption

In [0]:
for cat in categories:
    plot_init_model_charts(category = cat,
                           all_date_vals = output_data_df,
                           param_df = param_df)

-----------
# Additional Visualizations

## Single Time Series Plots
Same as above, but pulled out individually.

In [0]:
from tempo_forecasting.utils.plotting_utils import plot_time_series_demand, extract_series_from_date_vals
category_of_interest = "Car A"

In [0]:
data_series_dict = extract_series_from_date_vals(category_date_vals = output_data_df[(output_data_df["category"] == category_of_interest)])

# Train
plot_time_series_demand(title = f"Category: {category_of_interest}, Last CV Window", 
                        x_label = "Date", 
                        y_label = "Demand",
                        actual_dates = data_series_dict["actuals_full"]["date"], 
                        actual_vals = data_series_dict["actuals_full"]["vals"],
                        fitted_train_dates = data_series_dict["fitted_train"]["date"], 
                        fitted_train_vals = data_series_dict["fitted_train"]["vals"],
                        forecasted_test_dates = data_series_dict["forecasted_test"]["date"], 
                        forecasted_test_vals = data_series_dict["forecasted_test"]["vals"])

# Forecast
plot_time_series_demand(title = f"Category: {category_of_interest}, Final Forecast", 
                        x_label = "Date", 
                        y_label = "Demand",
                        actual_dates = data_series_dict["actuals_full"]["date"], 
                        actual_vals = data_series_dict["actuals_full"]["vals"],
                        fitted_train_dates = data_series_dict["refitted_train"]["date"], 
                        fitted_train_vals = data_series_dict["refitted_train"]["vals"],
                        forecasted_dates = data_series_dict["forecasted"]["date"], 
                        forecasted_vals = data_series_dict["forecasted"]["vals"])

## Deep Dive

It can be helpful to visualize different steps of the modeling process instead of taking Optuna's word that it has selected the best model. In order to do so, we must re-run a slightly modified version of the modified pipeline to capture a higher level of detail. The pipeline passes out time series data only for the best model's final cv window and final re-fit forecast. The code below re-runs modeling and saves time series values for every CV window of every model type, using that model type's best parameters.

In [0]:
from tempo_forecasting.utils.config_utils import get_models
from tempo_forecasting.utils.training_utils import cross_validate
from tempo_forecasting.utils.plotting_utils import core_plotter

category_of_interest = "Car A"

In [0]:
# Extract model details: models, associated classes, and associated parameters
category_cond = (results_df["category"] == category_of_interest)

coi_model_details = results_df[category_cond][["model","best_params"]]\
                            .set_index("model")\
                            .to_dict(orient="index")

model_classes = get_models(how="all")
for model_type in coi_model_details:
    coi_model_details[model_type]["model_class"] = model_classes[model_type]

# Extract modeling data
coi_modeling_data = output_data_df[(output_data_df["category"]==category_of_interest)&
                                   (~output_data_df["true_vals"].isna())][["date","true_vals"]]

In [0]:
desired_metrics = ["wmape","mae"]

# Do CV once (basically, mini train pipeline)
coi_all_results = {}
for model_type in coi_model_details.keys():
    cv_results = cross_validate(data = coi_modeling_data,      
                                date_col = "date", 
                                target_col = "true_vals", 
                                model_class = coi_model_details[model_type]["model_class"],
                                model_param_dict = coi_model_details[model_type]["best_params"], 
                                cv_dates = args["cv_dates"], 
                                metrics = desired_metrics)

    coi_all_results[model_type] = cv_results

# Run Eval Pipeline
eval_pipeline_output = {}
for model_type in coi_all_results.keys():
    data_dict = {
        "data_vals": np.array(coi_modeling_data["true_vals"]),
        "data_dates": np.array(coi_modeling_data["date"]),
        "train_test_split_dates":
            {
                "cv_dates": "date",
                "simple_train_test_dates": args["cv_dates"][-1]
            }
    }

    single_model_result = {
        "cv_metrics": {
            "cv_best_mean_all_metrics": {
                "WMAPE": np.round(np.mean([coi_all_results[model_type][i]["metrics"]["WMAPE"] 
                                for i in range(len(coi_all_results[model_type]))]),2)
            },
            "cv_best_full_all_metrics": {
                "WMAPE": [coi_all_results[model_type][i]["metrics"]["WMAPE"] 
                            for i in range(len(coi_all_results[model_type]))]
            },
        },
        "train_preds": coi_all_results[model_type][-1]["fitted_train_vals"],
        "test_preds": coi_all_results[model_type][-1]["test_pred_vals"],
        "model_params": coi_model_details[model_type]["best_params"]
    }

    eval_pipeline_input = {
        "models": {
            model_type : single_model_result
        },
        "data": data_dict
    }

    final_param_df, vals_df, logger = evaluation_pipeline(category = category_of_interest,
                                                        category_results = eval_pipeline_input,
                                                        args = args,
                                                        target_metric = "WMAPE",
                                                        forecast_horizon = 365,
                                                        logger = "print")
    
    eval_pipeline_output[model_type] = {
        "vals_df" : vals_df,
        "param_df" : final_param_df
    }

### Demonstration of Cross-Model Performance

Don't judge the poorly performing models below too hard - they would be doing much better if this example notebook gave them more than three Optuna trials.

In [0]:
chart_type = "train and pred" 
# or: train only

if chart_type == "train and pred":
    for model_type in eval_pipeline_output:
        plot_init_model_charts(category_of_interest,
                               eval_pipeline_output[model_type]["vals_df"],
                               eval_pipeline_output[model_type]["param_df"])

if chart_type == "train only":
    n_models = len(coi_all_results.keys())
    fig, axes = plt.subplots(n_models,1, sharex=True)
    fig.set_figheight(16)
    fig.set_figwidth(12)
    fig.subplots_adjust(top=0.95)
    fig.suptitle(f"{category_of_interest} - Model Comparison")

    i = 0

    for model_type in coi_all_results.keys():
        cv_min_date, cv_cutoff_date, cv_max_date = coi_all_results[model_type][-1]["dates"]
        fitted_train_date_cond = (coi_modeling_data["date"]>=cv_min_date)&(coi_modeling_data["date"]<=cv_cutoff_date)
        forecasted_test_date_cond = (coi_modeling_data["date"]>cv_cutoff_date)&(coi_modeling_data["date"]<=cv_max_date)

        all_dates = pd.to_datetime(coi_modeling_data["date"])

        mean_model_wmape = np.round(np.mean([coi_all_results[model_type][i]["metrics"]["WMAPE"] 
                                                for i in range(len(coi_all_results[model_type]))]),2)
        model_wmape = coi_all_results[model_type][-1]["metrics"]["WMAPE"]
        core_plotter(title = f"{model_type} model: final cv wmape = {model_wmape}, mean cv wmape = {mean_model_wmape}",
                                ax = axes[i],
                                x_label = "",
                                y_label = "Rental Demand",
                                actual_dates = all_dates,
                                actual_vals = coi_modeling_data["true_vals"],
                                fitted_train_dates = all_dates[fitted_train_date_cond],
                                fitted_train_vals = coi_all_results[model_type][-1]["fitted_train_vals"],
                                forecasted_test_dates = all_dates[forecasted_test_date_cond],
                                forecasted_test_vals = coi_all_results[model_type][-1]["test_pred_vals"])
        i += 1

### Demonstration of Walk Forward CV

In [0]:
models_of_interest = ["prophet"]

In [0]:
for model_type in models_of_interest:
    n_cv_windows = len(coi_all_results[model_type])
    fig, axes = plt.subplots(n_cv_windows,1, sharex=True)
    fig.set_figheight(16)
    fig.set_figwidth(12)
    fig.subplots_adjust(top=0.95)

    mean_model_wmape = np.round(np.mean([coi_all_results[model_type][i]["metrics"]["WMAPE"] 
                                            for i in range(len(coi_all_results[model_type]))]),2)
    fig.suptitle(f"{category_of_interest} - {model_type} model - CV Window Comparison (mean wmape = {mean_model_wmape})")


    for window_i in range(n_cv_windows):
        cv_min_date, cv_cutoff_date, cv_max_date = coi_all_results[model_type][window_i]["dates"]
        fitted_train_date_cond = (coi_modeling_data["date"]>=cv_min_date)&(coi_modeling_data["date"]<=cv_cutoff_date)
        forecasted_test_date_cond = (coi_modeling_data["date"]>cv_cutoff_date)&(coi_modeling_data["date"]<=cv_max_date)

        all_dates = pd.to_datetime(coi_modeling_data["date"])

        window_wmape = coi_all_results[model_type][window_i]["metrics"]["WMAPE"]
        core_plotter(title = f"Window {window_i+1}: {cv_min_date} to {cv_max_date}, wmape = {window_wmape}",
                     ax = axes[window_i],
                     x_label = "",
                     y_label = "Rental Demand",
                     actual_dates = all_dates,
                     actual_vals = coi_modeling_data["true_vals"],
                     fitted_train_dates = all_dates[fitted_train_date_cond],
                     fitted_train_vals = coi_all_results[model_type][window_i]["fitted_train_vals"],
                     forecasted_test_dates = all_dates[forecasted_test_date_cond],
                     forecasted_test_vals = coi_all_results[model_type][window_i]["test_pred_vals"])